# Investigating radiometer data

In this exercise, you will analyze data from a number of different radiometers and learn to determine the calibration coefficient of a pyranometer.

In [ ]:
# Install pvlib on Google Colab as this is not a standard package.
!pip install pvlib

In [2]:
import pvlib  # library for PV and solar calculations
import pandas as pd  # library for data analysis
import matplotlib.pyplot as plt  # library for 

## Step 1: Read dataset

The example dataset that we will be investigating is available [here](https://raw.githubusercontent.com/AdamRJensen/solar-resource-course/refs/heads/main/example_solar_radiation_dataset.csv). The dataset contains a subset of the measured parameters from DTU's solar radiometer station in Lyngby during the period 2025-08-26 to 2025-09-03.

The first step is to load the data into a pandas DataFrame:

In [8]:
url = 'https://raw.githubusercontent.com/AdamRJensen/solar-resource-course/refs/heads/main/example_solar_radiation_dataset.csv'

data = pd.read_csv(url, index_col=[0], parse_dates=[0])

data.head()

,CHP1_140049_DNI_mV,SMP22_200060_DHI_Wm2,CMP11_128767_GHI_mV,StarSchenk_7773_GHI_mV,solar_zenith,solar_azimuth
Time,,,,,,
2025-08-26 00:00:00+00:00,-0.002,-1.00,-0.014,-0.005,113.059,12.894
2025-08-26 00:01:00+00:00,-0.002,-1.00,-0.013,-0.005,113.027,13.159
2025-08-26 00:02:00+00:00,-0.002,-1.00,-0.013,-0.005,112.995,13.424
2025-08-26 00:03:00+00:00,-0.002,-1.01,-0.013,-0.005,112.962,13.689
2025-08-26 00:04:00+00:00,-0.002,-1.00,-0.013,-0.005,112.929,13.954


## Step 2: Convert analog signal to irradiance

Convert the analog signal of the following parameters to irradiance (W/m$^2$):
- CMP11_128767_GHI_mV
- CHP1_140049_DNI_mV

The calibration values can be found from the calibration sheet on Learn.

In [ ]:
# Update the code block

# Note the naming convention (the last part of the name denotes the unit)
df['CHP1_140049_DNI_Wm2'] = 

df['CMP11_128767_GHI_Wm2'] = 

## Step 3: Calculate GHI from measured DHI and DNI

In [ ]:
# Write your code here

## Step 4: Determine calibration coefficient for StarSchenk pyranometer

The dataset contains the analog output of an additional pyranometer (``StarSchenk_7773_GHI_mV``). The StarSchenk pyranometer was used by the Danish Meteorological Institute (DMI) from 2001 to 2024.

Since the calibration coefficient of the instrument is unknown, you have to determine this value by comparing it to the other high-quality measurements.

In [9]:
# Write your code here

## Step 5: Investigate the cosine response of the StartSchenk pyranometer

The cosine response of the pyranometer can be determined by plotting the calibration coefficient as a function of the solar zenith angle:

In [ ]:
# Write your code here